# Package Instalation

In [ ]:
!uv add langgraph langchain langsmith python-dotenv langchain_ollama langchain_community

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages:Annotated[list, add_messages]

### Integration with LLM using OLLAMA

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:latest", temperature=0)
llm

In [ ]:
## Node functionality
def chatbot(state:State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
graph_builder=StateGraph(State)

## Adding Node
graph_builder.add_node("llmchatbot", chatbot)

## Adding Edges
graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

## Compile the graph
graph=graph_builder.compile()

In [ ]:
## Visualize the graph
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

#### GET response from LLM

In [ ]:
response=graph.invoke({"messages":"Hello"})

for m in response["messages"]:
    m.pretty_print()

In [ ]:
response["messages"][-1].content

#### Stream LLM Response

In [ ]:
for event in graph.stream({"messages":"Hi, How are you?"}):
    print(event)